# Automatic generation of Notebook using PyCropML
    This notebook implements a crop model.

### Model Temp_profile

In [ ]:
MODULE list_sub
    IMPLICIT NONE
    TYPE container
        CLASS(*), ALLOCATABLE :: item
        CLASS(*), ALLOCATABLE :: items(:)
    END TYPE

    interface Add
        module procedure AddToListFloat
        module procedure AddToListInt
        module procedure AddToListChar
        module procedure AddToListIntArray
    end interface
CONTAINS

    FUNCTION indice(vectorElem, elem)
        CHARACTER(LEN=*), DIMENSION(:):: vectorElem
        INTEGER::iterator, indice
        CHARACTER(LEN=*):: elem
        DO iterator= 1, SIZE(vectorElem)
            IF(vectorElem(iterator)==elem) THEN
                indice = iterator
            END IF
        END DO
        RETURN
    END FUNCTION indice


    SUBROUTINE AddToListFloat(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        REAL, INTENT(IN) :: element
        REAL, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        REAL, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListFloat

    SUBROUTINE AddToListInt(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        INTEGER, INTENT(IN) :: element
        INTEGER, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        INTEGER, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListInt

    SUBROUTINE AddToListChar(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize, l
        CHARACTER(LEN=*), INTENT(IN) :: element
        CHARACTER(LEN=*), DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        CHARACTER(LEN=65), DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            l=1
            ALLOCATE(list(l))
            list(l) = element
        END IF
    END SUBROUTINE AddToListChar


    SUBROUTINE AddToListIntArray(a, e)
        INTEGER, ALLOCATABLE, INTENT(INOUT) :: a(:)
        INTEGER, ALLOCATABLE, INTENT(IN) :: e(:)
        INTEGER, ALLOCATABLE:: tmp(:)

        IF (.NOT.ALLOCATED(a)) THEN
            ALLOCATE(a(1))
            ALLOCATE(a(SIZE(e)), source = e)
        ELSE
            CALL MOVE_ALLOC(a,tmp)
            ALLOCATE(a(SIZE(tmp)+SIZE(e)), source = e)
            a(1:SIZE(tmp)) = tmp
            a(SIZE(tmp)+1:SIZE(a)) = e
        END IF
    END SUBROUTINE AddToListIntArray

END MODULE list_sub
MODULE Temp_profilemod
    USE list_sub
    IMPLICIT NONE
CONTAINS

    SUBROUTINE init_temp_profile(min_air_temp, &
        air_temp_day1, &
        layer_thick, &
        temp_amp, &
        prev_temp_profile, &
        prev_canopy_temp)
        IMPLICIT NONE
        INTEGER:: i_cyml_r
        REAL, INTENT(IN) :: min_air_temp
        REAL, INTENT(IN) :: air_temp_day1
        INTEGER , DIMENSION(: ), INTENT(IN) :: layer_thick
        REAL, INTENT(OUT) :: temp_amp
        REAL , DIMENSION(: ), INTENT(OUT) :: prev_temp_profile
        REAL, INTENT(OUT) :: prev_canopy_temp
        INTEGER:: soil_depth
        temp_amp = 0.0
        prev_canopy_temp = 0.0
        soil_depth = sum(layer_thick)
        allocate(prev_temp_profile(soil_depth))
        prev_temp_profile = air_temp_day1
        prev_canopy_temp = air_temp_day1
    END SUBROUTINE init_temp_profile

    SUBROUTINE model_temp_profile(temp_amp, &
        prev_temp_profile, &
        prev_canopy_temp, &
        min_air_temp, &
        air_temp_day1, &
        layer_thick, &
        temp_profile)
        IMPLICIT NONE
        INTEGER:: i_cyml_r
        REAL, INTENT(IN) :: temp_amp
        REAL , DIMENSION(: ), INTENT(IN) :: prev_temp_profile
        REAL, INTENT(IN) :: prev_canopy_temp
        REAL, INTENT(IN) :: min_air_temp
        REAL, INTENT(IN) :: air_temp_day1
        INTEGER , DIMENSION(: ), INTENT(IN) :: layer_thick
        REAL , DIMENSION(: ), INTENT(OUT) :: temp_profile
        INTEGER:: z
        INTEGER:: n
        REAL, ALLOCATABLE , DIMENSION(:):: vexp
        REAL:: therm_diff
        REAL:: temp_freq
        REAL:: therm_amp
        therm_diff = 5.37e-3
        temp_freq = 7.272e-5
        !- Name: temp_profile -Version: 1.0, -Time step: 1
        !- Description:
    !            * Title: tempprofile model
    !            * Authors: None
    !            * Reference: doi:http://dx.doi.org/10.1016/j.agrformet.2014.05.002
    !            * Institution: INRAE
    !            * ExtendedDescription: None
    !            * ShortDescription: None
        !- inputs:
    !            * name: temp_amp
    !                          ** description : current temperature amplitude
    !                          ** inputtype : variable
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLE
    !                          ** max : 100.0
    !                          ** min : 0.0
    !                          ** default : 0.0
    !                          ** unit : degC
    !            * name: prev_temp_profile
    !                          ** description : previous soil temperature profile 
    !                          ** inputtype : variable
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLEARRAY
    !                          ** len : 
    !                          ** max : 50.0
    !                          ** min : -50.0
    !                          ** default : 
    !                          ** unit : degC
    !            * name: prev_canopy_temp
    !                          ** description : previous crop temperature
    !                          ** inputtype : variable
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLE
    !                          ** max : 50.0
    !                          ** min : 0.0
    !                          ** default : 
    !                          ** unit : degC
    !            * name: min_air_temp
    !                          ** description : current minimum air temperature
    !                          ** inputtype : variable
    !                          ** variablecategory : exogenous
    !                          ** datatype : DOUBLE
    !                          ** max : 50.0
    !                          ** min : -50.0
    !                          ** default : 
    !                          ** unit : degC
    !            * name: air_temp_day1
    !                          ** description : Mean temperature on first day
    !                          ** inputtype : parameter
    !                          ** parametercategory : constant
    !                          ** datatype : DOUBLE
    !                          ** max : 100.0
    !                          ** min : 0.0
    !                          ** default : 0.0
    !                          ** unit : degC
    !            * name: layer_thick
    !                          ** description : layers thickness
    !                          ** inputtype : parameter
    !                          ** parametercategory : constant
    !                          ** datatype : INTARRAY
    !                          ** len : 
    !                          ** max : 
    !                          ** min : 
    !                          ** default : 
    !                          ** unit : cm
        !- outputs:
    !            * name: temp_profile
    !                          ** description : current soil profile temperature 
    !                          ** datatype : DOUBLEARRAY
    !                          ** variablecategory : state
    !                          ** len : 
    !                          ** max : 50.0
    !                          ** min : -50.0
    !                          ** unit : degC
        n = SIZE(prev_temp_profile)
        allocate(temp_profile(n))
        allocate(vexp(n))
        therm_amp = SQRT(temp_freq / 2 / therm_diff)
        DO z = 1 , n + 1-1, 1
            vexp(z - 1+1) = EXP(-z * therm_amp)
        END DO
        DO z = 1 , n + 1-1, 1
            temp_profile(z - 1+1) = prev_temp_profile(z - 1+1) - (vexp((z - 1)+1)  &
                    * (prev_canopy_temp - min_air_temp)) + (0.1 * (prev_canopy_temp -  &
                    prev_temp_profile(z - 1+1))) + (temp_amp * vexp((z - 1)+1) / 2)
        END DO
    END SUBROUTINE model_temp_profile

END MODULE
PROGRAM test
    USE Temp_profilemod

END PROGRAM
